# Imports

In [ ]:
import os

import plotting.plots
import bokeh.io
import bokeh.models
import bokeh.plotting
import bokeh.themes

import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import OMPython

import tables
import warnings
warnings.filterwarnings('ignore', category=tables.NaturalNameWarning)

import time

import scipy.io
import scipy.optimize
bokeh.io.output_notebook()

# Setup

In [ ]:
omc = OMPython.OMCSessionZMQ()
omc.sendExpression('getVersion()')

In [ ]:
font_size = '8pt'
theme  = {
    'attrs' : {
        'Plot': {
            'output_backend': 'svg',
        },
        'Axis': {
            'major_label_text_font_size': font_size,
            'axis_label_text_font_size': font_size,
            'minor_tick_in': 4,
            'minor_tick_out': 0,
            'major_tick_in': 8,
            'major_tick_out': 2,
        },
        'Grid': {
        },
        'Title': {
        },
        'Legend':{
            'label_text_font_size': '8pt',
            #'padding': 5
        },
        'BasicTickFormatter':{
            'precision': 2
        },
        'Line':{
            'line_width': 2
        }
    }
}

bokeh.plotting.curdoc().theme = bokeh.themes.Theme(json = theme)


In [ ]:
simulation_summary_filepath = './Simulation Datafiles/Simulation_Summaries.h5'

if os.path.isfile(simulation_summary_filepath):
    with pd.HDFStore(simulation_summary_filepath) as store:
        simulation_summaries = store['summary']
else:
    simulation_summaries = pd.DataFrame({'Simulation Class': pd.Series(dtype='str'),
                                         'Indentifier': pd.Series(dtype='str'),
                                         'resultFile': pd.Series(dtype='str'),
                                         'simulationOptions': pd.Series(dtype='str'),
                                         'timeFrontend': pd.Series(dtype='float'),
                                         'timeBackend': pd.Series(dtype='float'),
                                         'timeSimCode': pd.Series(dtype='float'),
                                         'timeTemplates': pd.Series(dtype='float'),
                                         'timeCompile': pd.Series(dtype='float'),
                                         'timeSimulation': pd.Series(dtype='float'),
                                         'timeTotal': pd.Series(dtype='float')})
    
    simulation_summaries = simulation_summaries.set_index(['Simulation Class', 'Indentifier'])
    simulation_summaries.to_hdf(simulation_summary_filepath, key = 'summary')

def save_results_properties(results_dict, class_name, identifier):
    simulation_summaries.loc[(class_name, identifier), :] = results_dict
    simulation_summaries.to_hdf(simulation_summary_filepath, key = 'summary')

# Helper Functions

In [ ]:

def transpose_char_array_to_string(array):
    return array.transpose().copy().view(f'<U{array.shape[0]}').squeeze(axis=1)
    

def parse_omc_mat(filepath, as_pandas = True):
    data = scipy.io.loadmat(filepath,
                            chars_as_strings = False
                           )

    data['name'] = transpose_char_array_to_string(data['name'])
    data['description'] = transpose_char_array_to_string(data['description'])

    series_dict = {}
    parameters = {}
    for i, name in enumerate(data['name']):
        if data['dataInfo'][0, i] in [0,2]:
            #We have a time series variable
            index = data['dataInfo'][1, i]
            series_dict[name] =  np.sign(index)*data['data_2'][np.abs(index)-1]
        else:
            index = data['dataInfo'][1, i]
            parameters[name] =  np.sign(index)*data['data_1'][np.abs(index)-1][0]
    
    if as_pandas:
        timeseries = pd.DataFrame.from_dict(series_dict).set_index('time')
    else:
        timeseries = series_dict
        
    return timeseries, parameters


In [ ]:
def reload_omc():
    omc.sendExpression("clear()")
    omc.sendExpression('cd("./Simulation_Temp/")')
    omc.loadFile('../Modelica/Ionics 0.1.0.om/package.mo')
    omc.loadFile('../Modelica/LiteratureComparison/package.mo')
    omc.loadFile('../Modelica/NovelCircuitry/package.mo')

# Run Simulations

## Berggren

### Single Diode

In [ ]:
reload_omc()

#### Optimization

#### Simulate Literature Trials

In [ ]:
# Get default values for parameters
reverse_duration = float(omc.sendExpression('getParameterValue(LiteratureComparison.Berggren.Experiments.Diode_steps, "reverse_duration")'))
forward_duration = float(omc.sendExpression('getParameterValue(LiteratureComparison.Berggren.Experiments.Diode_steps, "forward_duration")'))
equilibration_duration = float(omc.sendExpression('getParameterValue(LiteratureComparison.Berggren.Experiments.Diode_steps, "equilibration_duration")'))
transition_duration = float(omc.sendExpression('getParameterValue(LiteratureComparison.Berggren.Experiments.Diode_steps, "transition_duration")'))

In [ ]:
forward_durations = [5, 10, 20, 30, 60, 90]
#forward_durations = [90]

results = {}

#x = optimized_solution.x
#x = [1.506e-3, 2e-1, 30e-6, 0.4e-6] #30 micron PEDOT:PSS

# constant Real D_multiplier_polyanion = 2e-3;
# constant Real D_multiplier_polycation = 1.0;

# parameter Real film_thickness_polyanion = 9.0e-6;
# parameter Real film_thickness_polycation = 0.1e-6;

#x = [0.84e-3, 2e-1, 60e-6, 0.4e-6] #60 micron PEDOT:PSS Old best fit
#x = [2.0e-3, 4.5e-1, 9.0e-6, 0.2e-6] #9 micron PEDOT:PSS, charge 4000 1000
x = [4.0e-1, 4.0e-3, 0.3e-6, 5.0e-6] #300 nanometer PEDOT:PSS, charge 1000 3000

D_multiplier_polyanion = x[0];
D_multiplier_polycation = x[1];
film_thickness_polyanion = x[2];
film_thickness_polycation = x[3];

reverse_duration = 120   
omc.sendExpression(f'setParameterValue(LiteratureComparison.Berggren.Experiments.Diode_steps, reverse_duration, {reverse_duration})')
    
#omc.sendExpression(f'setElementModifierValue(LiteratureComparison.Berggren.Experiments.Diode_steps, diode.D_multiplier_polyanion, $Code(={D_multiplier_polyanion}))')
#omc.sendExpression(f'setElementModifierValue(LiteratureComparison.Berggren.Experiments.Diode_steps, diode.D_multiplier_polycation, $Code(={D_multiplier_polycation}))')
#omc.sendExpression(f'setElementModifierValue(LiteratureComparison.Berggren.Experiments.Diode_steps, diode.film_thickness_polyanion, $Code(={film_thickness_polyanion}))')
#omc.sendExpression(f'setElementModifierValue(LiteratureComparison.Berggren.Experiments.Diode_steps, diode.film_thickness_polycation, $Code(={film_thickness_polycation}))')

filepath = './Simulation Datafiles/Berggren_Single_Diode_Duration_Sweep.h5'

with pd.HDFStore(filepath, mode='w') as store:
    for forward_duration in forward_durations:
    
        print(f'Simulating duration {forward_duration}')

        class_name = 'LiteratureComparison.Berggren.Experiments.Diode_steps'
        
        omc.sendExpression(f'setParameterValue({class_name}, forward_duration, {forward_duration})')
    
        time_stop = equilibration_duration + reverse_duration*2 + forward_duration

        flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian --generateSymbolicJacobian'
        flags_sim = '-noEquidistantTimeGrid -nls=kinsol -w -lv=LOG_STDOUT,LOG_ASSERT,LOG_EVENTS,LOG_STATS -jacobian=coloredSymbolical'
    
        result_dict = omc.sendExpression(f'simulate({class_name}, stopTime={time_stop}, options="{flags_compile}", simflags="{flags_sim}")')
        
        save_results_properties(result_dict,
                                class_name,
                                f'Duration = {forward_duration}'
                               )
        
        results[forward_duration] = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

        store[f'Simulation_Duration_{forward_duration}/timeseries'] = results[forward_duration][0]
        store[f'Simulation_Duration_{forward_duration}/parameters'] = pd.DataFrame.from_dict(results[forward_duration][1], orient='index', columns=['value'])

        node = store.get_node(f'Simulation_Duration_{forward_duration}')
        tables.Array(node, 'forward_duration', forward_duration)
    
    

### Full Wave Rectifier

In [ ]:
reload_omc()

In [ ]:
#results = {}

filepath = './Simulation Datafiles/Berggren_Full_Wave.h5'

with pd.HDFStore(filepath) as store:

    class_name = 'LiteratureComparison.Berggren.Experiments.FullRectifier'

    flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian --generateSymbolicJacobian'
    flags_sim = '-noEquidistantTimeGrid -nls=kinsol -w -lv=LOG_STDOUT,LOG_ASSERT,LOG_EVENTS,LOG_STATS -jacobian=coloredSymbolical'
    
    result_dict = omc.sendExpression(f'simulate({class_name}, options="{flags_compile}", simflags="{flags_sim}")')
    save_results_properties(result_dict,
                        class_name,
                        ''
                       )

    results = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')


    
    store[f'Simulation/timeseries'] = results[0]
    store[f'Simulation/parameters'] = pd.DataFrame.from_dict(results[1], orient='index', columns=['value'])

    

## Yossifon

### Single Diode

#### Optimization

#### Literature Trial

In [ ]:
reload_omc()

In [ ]:
# Get default values for parameters
reverse_duration = float(omc.sendExpression('getParameterValue(LiteratureComparison.Berggren.Experiments.Diode_steps, "reverse_duration")'))
forward_duration = float(omc.sendExpression('getParameterValue(LiteratureComparison.Berggren.Experiments.Diode_steps, "forward_duration")'))
equilibration_duration = float(omc.sendExpression('getParameterValue(LiteratureComparison.Berggren.Experiments.Diode_steps, "equilibration_duration")'))
transition_duration = float(omc.sendExpression('getParameterValue(LiteratureComparison.Berggren.Experiments.Diode_steps, "transition_duration")'))

In [ ]:
#results = {}

filepath = './Simulation Datafiles/Yossifon_Single_Diode.h5'

with pd.HDFStore(filepath) as store:
    
    class_name = 'LiteratureComparison.Yossifon.Experiments.Diode_steps'
    flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian --generateSymbolicJacobian '
    flags_sim = '-noEquidistantTimeGrid -nls=kinsol -w -lv=LOG_STDOUT,LOG_ASSERT,LOG_EVENTS,LOG_STATS -jacobian=coloredSymbolical'
    
    result_dict = omc.sendExpression(f'simulate({class_name}, options="{flags_compile}", simflags="{flags_sim}")')
    
    save_results_properties(result_dict,
                            class_name,
                            ''
                           )

    results = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

    store[f'Simulation/timeseries'] = results[0]
    store[f'Simulation/parameters'] = pd.DataFrame.from_dict(results[1], orient='index', columns=['value'])

    

### AND Gate

In [ ]:
reload_omc()

#### Without Leakage

In [ ]:
#results = {}

filepath = './Simulation Datafiles/Yossifon_AND_Gate.h5'
class_name = 'LiteratureComparison.Yossifon.Experiments.Gate_AND_NoLeak_Steps'
flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian --generateSymbolicJacobian '
flags_sim = '-noEquidistantTimeGrid -nls=kinsol -w -lv=LOG_STDOUT,LOG_ASSERT,LOG_EVENTS,LOG_STATS -jacobian=coloredSymbolical'
    
result_dict = omc.sendExpression(f'simulate({class_name}, options="{flags_compile}", simflags="{flags_sim}")')
save_results_properties(result_dict,
                        class_name,
                        ''
                       )

results = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

with pd.HDFStore(filepath) as store:    
    store[f'Simulation_No_Leak/timeseries'] = results[0]
    store[f'Simulation_No_Leak/parameters'] = pd.DataFrame.from_dict(results[1], orient='index', columns=['value'])
    

#### With Leakage

##### Optimization

##### Literature Trial


In [ ]:
reload_omc()

In [ ]:
filepath = './Simulation Datafiles/Yossifon_AND_Gate.h5'
class_name = 'LiteratureComparison.Yossifon.Experiments.Gate_AND_Leak_Steps'
flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian --generateSymbolicJacobian '
flags_sim = '-noEquidistantTimeGrid -nls=kinsol -w -lv=LOG_STDOUT,LOG_ASSERT,LOG_EVENTS,LOG_STATS -jacobian=coloredSymbolical'
    
result_dict = omc.sendExpression(f'simulate({class_name}, options="{flags_compile}", simflags="{flags_sim}")')
save_results_properties(result_dict,
                        class_name,
                        ''
                       )

results = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')
with pd.HDFStore(filepath) as store:
    store[f'Simulation_Leak/timeseries'] = results[0]
    store[f'Simulation_Leak/parameters'] = pd.DataFrame.from_dict(results[1], orient='index', columns=['value'])


## Soft Robot Circuit

In [ ]:
reload_omc()

#### Concentration Sweep

In [ ]:
concentrations = np.logspace(0, 3.5, 20)

filepath_output = './Simulation Datafiles/Robot_Brain.h5'
flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian --generateSymbolicJacobian '
flags_sim = '-port=12345 -override=startTime=0,stopTime=70000,stepSize=10,tolerance=1e-08,solver=dassl,outputFormat=mat -jacobian=coloredSymbolical'

for c in concentrations:
    omc.sendExpression(f"setParameterValue(NovelCircuitry.Robot_Brain, bath_concentration_world, {{{c}, {c}}})")
    filename_prefix = f"NovelCircuitry.Robot_Brain_{c}"
    result_message = omc.sendExpression(f'simulate(NovelCircuitry.Robot_Brain, fileNamePrefix="{filename_prefix}", options="{flags_compile}", simflags="{flags_sim}")')
    result = parse_omc_mat(f'Simulation_Temp/{filename_prefix}_res.mat')

    with pd.HDFStore(filepath_output) as store:
        store[f'{c}/timeseries'] = result[0]
        store[f'{c}/parameters'] = pd.DataFrame.from_dict(result[1], orient='index', columns=['value'])


#### Dynamic Concentration

In [ ]:
filepath_output = './Simulation Datafiles/Robot_Brain_Dynamic.h5'
class_name = 'NovelCircuitry.Robot_Brain_Changing_Concentration'
flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian --generateSymbolicJacobian '
flags_sim = '-port=12345 -override=startTime=0,stopTime=32000,stepSize=1,tolerance=1e-08,solver=dassl,outputFormat=mat -jacobian=coloredSymbolical'

result_dict = omc.sendExpression(f'simulate({class_name}, options="{flags_compile}", simflags="{flags_sim}")')
save_results_properties(result_dict,
                        class_name,
                        ''
                       )

result = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

with pd.HDFStore(filepath_output) as store:
    store[f'timeseries'] = result[0]
    store[f'parameters'] = pd.DataFrame.from_dict(result[1], orient='index', columns=['value'])


### Transistor

In [ ]:
class_name = 'NovelCircuitry.Oscillator_Single_Transistor_Output_Curve_Trace'
filepath_output = './Simulation Datafiles/Robot_Brain_Transistor_Oscillator_Output_Characteristics.h5'

result_dict = omc.sendExpression(f'simulate({class_name}, simflags="-port=12345 -override=startTime=0,stopTime=6000,stepSize=1,tolerance=1e-08,solver=dassl,outputFormat=mat -jacobian=coloredSymbolical")')

save_results_properties(result_dict,
                        class_name,
                        ''
                       )

result = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

with pd.HDFStore(filepath_output) as store:
    store[f'timeseries'] = result[0]
    store[f'parameters'] = pd.DataFrame.from_dict(result[1], orient='index', columns=['value'])


In [ ]:
class_name = 'NovelCircuitry.Oscillator_Single_Transistor_Response'
filepath_output = './Simulation Datafiles/Robot_Brain_Transistor_Oscillator_Current_Amplification.h5'

result_dict = omc.sendExpression(f'simulate({class_name}, simflags="-port=12345 -override=startTime=0,stopTime=6000,stepSize=1,tolerance=1e-08,solver=dassl,outputFormat=mat -jacobian=coloredSymbolical")')
save_results_properties(result_dict,
                        class_name,
                        ''
                       )
result = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

with pd.HDFStore(filepath_output) as store:
    store[f'timeseries'] = result[0]
    store[f'parameters'] = pd.DataFrame.from_dict(result[1], orient='index', columns=['value'])


In [ ]:
class_name = 'NovelCircuitry.Actuator_Single_Transistor_Output_Curve_Trace'
filepath_output = './Simulation Datafiles/Robot_Brain_Transistor_Actuator_Output_Characteristics.h5'

result_dict = omc.sendExpression(f'simulate({class_name}, simflags="-port=12345 -override=startTime=0,stopTime=36000,stepSize=1,tolerance=1e-08,solver=dassl,outputFormat=mat -jacobian=coloredNumerical")')
save_results_properties(result_dict,
                        class_name,
                        ''
                       )
result = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

with pd.HDFStore(filepath_output) as store:
    store[f'timeseries'] = result[0]
    store[f'parameters'] = pd.DataFrame.from_dict(result[1], orient='index', columns=['value'])


In [ ]:
class_name = 'NovelCircuitry.Actuator_Single_Transistor_Response'
filepath_output = './Simulation Datafiles/Robot_Brain_Transistor_Actuator_Current_Amplification.h5'

result_message = omc.sendExpression(f'simulate({class_name}, simflags="-port=12345 -override=startTime=0,stopTime=12000,stepSize=1,tolerance=1e-08,solver=dassl,outputFormat=mat -jacobian=coloredNumerical")')
save_results_properties(result_dict,
                        class_name,
                        ''
                       )
result = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

with pd.HDFStore(filepath_output) as store:
    store[f'timeseries'] = result[0]
    store[f'parameters'] = pd.DataFrame.from_dict(result[1], orient='index', columns=['value'])


## Demonstration Circuits

### Wire Enrichment

In [ ]:
reload_omc()

In [ ]:
#results = {}

filepath = './Simulation Datafiles/Wire_Enrichment.h5'

with pd.HDFStore(filepath) as store:
    
    class_name = 'NovelCircuitry.WireEnrichment'
    flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian --generateSymbolicJacobian '
    flags_sim = '-noEquidistantTimeGrid -nls=kinsol -w -lv=LOG_STDOUT,LOG_ASSERT,LOG_EVENTS,LOG_STATS -jacobian=coloredSymbolical'
    
    result_dict = omc.sendExpression(f'simulate({class_name}, options="{flags_compile}", simflags="{flags_sim}")')
    
    save_results_properties(result_dict,
                            class_name,
                            ''
                           )

    results = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

    store[f'timeseries'] = results[0]
    store[f'parameters'] = pd.DataFrame.from_dict(results[1], orient='index', columns=['value'])


filepath = './Simulation Datafiles/Wire_Enrichment_Uncharged.h5'

with pd.HDFStore(filepath) as store:
    
    class_name = 'NovelCircuitry.WireEnrichmentBaseCase'
    flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian --generateSymbolicJacobian '
    flags_sim = '-noEquidistantTimeGrid -nls=kinsol -w -lv=LOG_STDOUT,LOG_ASSERT,LOG_EVENTS,LOG_STATS -jacobian=coloredSymbolical'
    
    result_dict = omc.sendExpression(f'simulate({class_name}, options="{flags_compile}", simflags="{flags_sim}")')
    
    save_results_properties(result_dict,
                            class_name,
                            ''
                           )

    results = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

    store[f'timeseries'] = results[0]
    store[f'parameters'] = pd.DataFrame.from_dict(results[1], orient='index', columns=['value'])
    

### Warburg element

In [ ]:
reload_omc()

In [ ]:
import scipy.interpolate

def zero_crossings(x,y):
    return scipy.interpolate.CubicSpline(x,y).roots(extrapolate=False)

def get_impedance(i,v,t,hz):
    zeros_i_t = zero_crossings(t, i)
    zeros_v_t = zero_crossings(t, v)

    # Skip the first period
    zeros_i_t = zeros_i_t[zeros_i_t > 0.01/hz]
    zeros_v_t = zeros_v_t[zeros_v_t > 0.01/hz]

    min_length = min(len(zeros_i_t), len(zeros_v_t))

    # Skip two more crossings
    delta_t = zeros_i_t[2:min_length] - zeros_v_t[2:min_length]
    
    shift = (np.mean(delta_t)*hz)*(2*np.pi)

    # Compute amplitude from the second half
    amplitude = np.max(v[len(v)//2:])/np.max(i[len(i)//2:])

    #set_trace()

    return amplitude*np.exp(1j*shift)
    # return (amplitude, shift)

In [ ]:
#results = {}

filepath = './Simulation Datafiles/Warburg element.h5'

frequencies = np.logspace(-4,3,50)
# frequencies = frequencies[0]

with pd.HDFStore(filepath) as store:
    
    class_name = 'NovelCircuitry.warburg'
    flags_compile = '--matchingAlgorithm=PFPlusExt --indexReductionMethod=dynamicStateSelection -d=initialization,evaluateAllParameters,NLSanalyticJacobian  --generateDynamicJacobian=symbolic'
    flags_sim = '-nls=kinsol -w -lv=LOG_STDOUT,LOG_ASSERT,LOG_EVENTS,LOG_STATS -jacobian=coloredSymbolical'

    for frequency in frequencies:

        print(frequency)
    
        omc.sendExpression(f'setElementModifierValue(NovelCircuitry.warburg, sineVoltage.f, $Code(={frequency:0.6e}))')
        result_dict = omc.sendExpression(f'simulate({class_name}, numberOfIntervals = 10000, stopTime={6/frequency:0.6e}, options="{flags_compile}", simflags="{flags_sim}")')
        
        save_results_properties(result_dict,
                                class_name,
                                ''
                               )
    
        results = parse_omc_mat(f'Simulation_Temp/{class_name}_res.mat')

        timeseries = results[0]
        timeseries = timeseries[~timeseries.index.duplicated(keep='first')]


        key = f'timeseries {frequency:0.4e}'
        store[key] = timeseries
        store.get_storer(key).attrs.frequency = frequency 


In [ ]:
impedances = {}
frequencies = np.logspace(-4,3,50)

with pd.HDFStore(filepath) as store:
    for frequency in frequencies:
        print(frequency)
        
        key = f'timeseries {frequency:0.4e}'
        timeseries = store[key]
        
        i = -timeseries['sineVoltage.i']
        v = timeseries['sineVoltage.v']
        
        t = i.index
        
        impedances[frequency] = get_impedance(i.to_numpy(), v.to_numpy(), t.to_numpy(), frequency)

        #print(t.max())
        # print(len(t))
        
        #store[f'timeseries'] = results[0]
        #store[f'parameters'] = pd.DataFrame.from_dict(results[1], orient='index', columns=['value'])

impedances = np.array(list(impedances.values()))

In [ ]:
# plt.figure()
# plt.plot(np.log10(frequencies), np.log10(np.abs(impedances)))
# plt.figure()
# plt.plot(np.log10(frequencies), np.angle(impedances)*360/(2*np.pi))


plot = plotting.plots.Bode(width = 400, height = 400)
plot.add_trace(frequency=frequencies, magnitude=np.abs(impedances), angle=np.angle(impedances)*180/np.pi)
bokeh.io.show(plot.layout)

bokeh.io.export_svg(plot.layout,
                    filename = './Figures/Warburg/bode.svg',
                    webdriver=driver)


plot = plotting.plots.Columns([{'x': 'real', 'y': ['imag']}], {'real': "Z'"}, {'imag': "-Z''"},
                             height=400, width=400)
plot.add_trace(real=np.real(impedances), imag=-np.imag(impedances))
bokeh.io.show(plot.layout)

bokeh.io.export_svg(plot.layout,
                    filename = './Figures/Warburg/nyquist.svg',
                    webdriver=driver)


# Figures

In [ ]:
style_line_experiment = {'line_width': 2, 
                 'line_dash': [4,4],
                 'line_cap': 'butt',
                 'line_join': 'round',
                 'line_alpha': 1}

style_line_simulation = {'line_width': 2,
                 'line_alpha': 0.6,}

colors = bokeh.palettes.Category10_10

## Single Diode

### Berggren

In [ ]:
# Read in the simulation data from the saved file

filepath = './Simulation Datafiles/Berggren_Single_Diode_Duration_Sweep.h5'

data_timeseries = {}
forward_durations = {}

with pd.HDFStore(filepath) as store:
    root = store.get_node('/')
    
    for node_key in store.keys():

        node_key_parts = node_key.split('/')

        if node_key_parts[2] == 'timeseries':
            data_timeseries[node_key_parts[1]] = store[node_key]

        print(f'Reading node {node_key}')

        forward_durations[node_key_parts[1]] = store.get_node(f'/{node_key_parts[1]}/forward_duration').read()
        #Computational_costs[node_key_parts[1]] = 

diode_currents = {key: data_timeseries[key]['diode.i'] for key in data_timeseries}
times = {key: data_timeseries[key].reset_index()['time'] for key in data_timeseries}

In [ ]:
# Read in literature values:
data_berggren = pd.read_excel('./Literature Data/Berggren Single Diode/fig3.xlsx',
                              header=[0,1],
                              index_col = 0,
                              decimal=',')

In [ ]:
class DiodeComparison(plotting.plots.Columns):
    def _plot_trace(self, column_data_source, key_data, key_figure, style, color_trace):
        
        if 'type' in style:
            type = style.pop('type')
        else: 
            type = None
        
        if type == 'experiment':
            style_applied = {'color': color_trace, 
                             'legend_label': 'Experiment'} | style_line_experiment | style
            
            glyph = self.figures[key_figure].line(
                source=column_data_source,
                x=key_data[0],
                y=key_data[1],
                **style_applied,
            )
        else:
            style_applied = {'color': color_trace, 'legend_label': 'Simulation'} | style_line_simulation | style
            
            glyph = self.figures[key_figure].line(
                source=column_data_source,
                x=key_data[0],
                y=key_data[1],
                line_join = 'round',
                **style_applied,
            )
        
        return [glyph]

plot = DiodeComparison(arrangement=[{'x':'time',
                                            'y':['diode_current']}],
                              x_labels={'time': " "},
                              y_labels = {'diode_current': " "},
                              height = 276,
                              width = 340)
reverse_duration = 120
equilibration_duration = 30

for key in data_timeseries:
    print(key)
    time_offset = equilibration_duration + reverse_duration + forward_durations[key] - 0.25
    plot.add_trace(time=times[key] - time_offset,
                   diode_current=diode_currents[key]/1e-6,
                   label=forward_durations[key],)

colors_experiment = ['#2ca02c', '#8c564b', '#9467bd', '#d62728', '#ff7f0e', '#1f77b4']

plot.colors = plotting.plots.itertools.cycle(colors_experiment)

for column in data_berggren['BM1P']:
    if 'Current' not in column:
        continue

    print(column)
    data = bokeh.models.ColumnDataSource({'time': data_berggren.index, 'diode_current': data_berggren['BM1P', column]/1e-6})
    plot.add_trace(column_data_source=data,
                   label=key,
                   style={'type': 'experiment'})


In [ ]:
plot.figures['time', 'diode_current'].y_range.start = -0.8
plot.figures['time', 'diode_current'].y_range.end = 0.5
plot.figures['time', 'diode_current'].x_range.start = -100
plot.figures['time', 'diode_current'].x_range.end = 70
#plot.figures['time', 'diode_current'].yaxis.formatter.precision = 2
plot.figures['time', 'diode_current'].legend.padding = 5
plot.figures['time', 'diode_current'].legend.spacing = -5
plot.figures['time', 'diode_current'].output_backend = 'svg'


In [ ]:
bokeh.io.show(plot.layout)

### Yossifon

In [ ]:
# Read in literature values:
data_yossifon = pd.read_csv('./Literature Data/Yossifon Single Diode/Current vs Time.csv', names=['time', 'current'])

In [ ]:
#results = {}

filepath = './Simulation Datafiles/Yossifon_Single_Diode.h5'

with pd.HDFStore(filepath) as store:

    data_timeseries = store[f'Simulation/timeseries']
    data_parameters = store[f'Simulation/parameters']


In [ ]:
plot = DiodeComparison(arrangement=[{'x':'time',
                                            'y':['diode_current']}],
                              x_labels={'time': " "},
                              y_labels = {'diode_current': " "},
                              height = 276,
                              width = 340)

plot.add_trace(time=data_timeseries.reset_index()['time']-2200,
                   diode_current=data_timeseries['diode.i']/1e-6,
                   label='Simulation',
                   style={'line_color':'black'})

plot.add_trace(time = data_yossifon['time']-1400,
               diode_current = data_yossifon['current']/1e-6,
               label=key,
               style={'type': 'experiment', 'line_color':'black'})

plot.figures['time', 'diode_current'].y_range.start = -3.5
plot.figures['time', 'diode_current'].y_range.end = 2.5
plot.figures['time', 'diode_current'].x_range.start = 320-600
plot.figures['time', 'diode_current'].x_range.end = 800-600
plot.figures['time', 'diode_current'].yaxis.formatter.precision = 0
plot.figures['time', 'diode_current'].legend.padding = 5
plot.figures['time', 'diode_current'].legend.spacing = -5
plot.figures['time', 'diode_current'].output_backend = 'svg'

bokeh.io.show(plot.layout)

## Full Wave Rectifier

In [ ]:
# Read in literature values:
data_berggren_full_wave_current_in = pd.read_csv('./Literature Data/Berggren Full Wave Rectifier/I_in.csv', names=['time', 'current_in'])
data_berggren_full_wave_current_out = pd.read_csv('./Literature Data/Berggren Full Wave Rectifier/I_out.csv', names=['time', 'current_out'])

data_berggren_full_wave = pd.merge(data_berggren_full_wave_current_in, data_berggren_full_wave_current_out, 'outer')


In [ ]:
#results = {}

filepath = './Simulation Datafiles/Berggren_Full_Wave.h5'

with pd.HDFStore(filepath) as store:

    data_timeseries = store[f'Simulation/timeseries']
    data_parameters = store[f'Simulation/parameters']


In [ ]:
plot_current = bokeh.plotting.figure(height = 170, width = 285)

plot_current.line(x=data_timeseries.reset_index()['time'],
                  y=data_timeseries['load.i']/1e-6,
                  **style_line_simulation,
                  legend_label = 'Simulation',
                  color = 'black'
                 )

plot_current.line(x = data_berggren_full_wave['time'],
                  y = data_berggren_full_wave['current_out']/1e-6,
                  **style_line_experiment,
                  legend_label = 'Experiment',
                  color='black'
              )

plot_voltage = bokeh.plotting.figure(height = 50, width = 285)
plot_voltage.x_range = plot_current.x_range


plot_voltage.line(x=data_timeseries.reset_index()['time'],
                  y=data_timeseries['trapezoidVoltage.v'],
                  **style_line_simulation,
                  color = 'black'
                 )

layout = bokeh.layouts.gridplot(children = [[plot_voltage],
                                          [plot_current]],
                               )

In [ ]:
plot_current.y_range.start = -5e-1
plot_current.y_range.end = 4.5e-1
plot_current.x_range.start = -30
plot_current.x_range.end = 1000
plot_current.legend.location = 'bottom_right'
plot_current.legend.padding = 5
plot_current.legend.spacing = -5
plot_current.output_backend = 'svg'
plot_current.yaxis.formatter.precision = 2

plot_voltage.yaxis.ticker = [-4,0,4]
plot_voltage.xaxis.visible = False
plot_voltage.y_range.start = -5
plot_voltage.y_range.end = 5
plot_voltage.output_backend = 'svg'


In [ ]:
bokeh.io.show(layout)

## AND Gate

In [ ]:
# Read in literature values:
data_yossifon_and_gate_voltage= pd.read_csv('./Literature Data/Yossifon AND Gate/Current.csv', names=['time', 'voltage'])


In [ ]:
#results = {}

filepath = './Simulation Datafiles/Yossifon_AND_Gate.h5'

with pd.HDFStore(filepath) as store:
    
    data_no_leak_timeseries = store[f'Simulation_No_Leak/timeseries']
    data_no_leak_parameters = store[f'Simulation_No_Leak/parameters']

    data_leak_timeseries = store[f'Simulation_Leak/timeseries']
    data_leak_parameters = store[f'Simulation_Leak/parameters']

In [ ]:
plot_voltage_out = bokeh.plotting.figure(height = 190, width = 278)

plot_voltage_out.line(x=data_no_leak_timeseries.reset_index()['time'],
                  y=data_no_leak_timeseries['gate.volume_center.port.V'],
                  **style_line_simulation,
                  legend_label = 'Simulation',
                  color = 'black'
                 )

plot_voltage_out.line(x = data_yossifon_and_gate_voltage['time'],
                  y = data_yossifon_and_gate_voltage['voltage'],
                  **style_line_experiment,
                  legend_label = 'Experiment',
                  color='black'
              )

plot_voltage_in = bokeh.plotting.figure(height = 50, width = 278)
plot_voltage_in.x_range = plot_voltage_out.x_range

plot_voltage_in.line(x=data_no_leak_timeseries.reset_index()['time'],
                  y=data_no_leak_timeseries['source_a.v'],
                  **style_line_simulation,
                  color = '#1f77b4'
                 )

plot_voltage_in.line(x=data_no_leak_timeseries.reset_index()['time'],
                  y=data_no_leak_timeseries['source_b.v'],
                  **style_line_simulation,
                  color = '#ff7f0eff'
                 )

layout = bokeh.layouts.gridplot(children = [[plot_voltage_in],
                                          [plot_voltage_out]],
                               )

In [ ]:
plot_voltage_out.y_range.start = -0.1
plot_voltage_out.y_range.end = 1.1
#plot.figures['time', 'diode_current'].x_range.start = -5
plot_voltage_out.x_range.end = 1200
plot_voltage_out.legend.location = 'top_right'
plot_voltage_out.legend.padding = 5
plot_voltage_out.legend.spacing = -5
plot_voltage_out.output_backend = 'svg'

plot_voltage_in.yaxis.ticker = [0,1]
plot_voltage_in.xaxis.visible = False
plot_voltage_in.y_range.start = -0.5
plot_voltage_in.y_range.end = 1.5
plot_voltage_in.output_backend = 'svg'

In [ ]:
bokeh.io.show(layout)

In [ ]:
# Read in literature values:
data_yossifon_and_gate_voltage= pd.read_csv('./Literature Data/Yossifon AND Gate/Current.csv', names=['time', 'voltage'])


## AND Gate with Leakage

In [ ]:
#results = {}

filepath = './Simulation Datafiles/Yossifon_AND_Gate.h5'

with pd.HDFStore(filepath) as store:
    
    data_no_leak_timeseries = store[f'Simulation_No_Leak/timeseries']
    data_no_leak_parameters = store[f'Simulation_No_Leak/parameters']

    data_leak_timeseries = store[f'Simulation_Leak/timeseries']
    data_leak_parameters = store[f'Simulation_Leak/parameters']

In [ ]:
plot_voltage_out = bokeh.plotting.figure(height = 300, width = 600)

colors = bokeh.palettes.Category10_10

plot_voltage_out.line(x=data_leak_timeseries.reset_index()['time']-800,
                  y=data_leak_timeseries['gate.volume_center.port.V'],
                  **style_line_simulation,
                  legend_label = 'Simulation With Leakage',
                  color = colors[2]
                 )

plot_voltage_out.line(x=data_no_leak_timeseries.reset_index()['time'],
                  y=data_no_leak_timeseries['gate.volume_center.port.V'],
                  **style_line_simulation,
                  legend_label = 'Simulation Without Leakage',
                  color = colors[4]
                 )

plot_voltage_out.line(x = data_yossifon_and_gate_voltage['time'],
                  y = data_yossifon_and_gate_voltage['voltage'],
                  **style_line_experiment,
                  legend_label = 'Experiment',
                  color='black'
              )

plot_voltage_in = bokeh.plotting.figure(height = 50, width = 600)
plot_voltage_in.x_range = plot_voltage_out.x_range

plot_voltage_in.line(x=data_leak_timeseries.reset_index()['time']-800,
                  y=data_leak_timeseries['source_a.v'],
                  **style_line_simulation,
                  color = '#1f77b4'
                 )

plot_voltage_in.line(x=data_leak_timeseries.reset_index()['time']-800,
                  y=data_leak_timeseries['source_b.v'],
                  **style_line_simulation,
                  color = '#ff7f0eff'
                 )

layout = bokeh.layouts.gridplot(children = [[plot_voltage_in],
                                          [plot_voltage_out]],
                               )

In [ ]:
plot_voltage_out.y_range.start = -0.1
plot_voltage_out.y_range.end = 1.15
#plot.figures['time', 'diode_current'].x_range.start = -5
plot_voltage_out.x_range.end = 1200
plot_voltage_out.x_range.start = -50
plot_voltage_out.legend.location = 'top_right'
plot_voltage_out.legend.padding = 5
plot_voltage_out.legend.spacing = -5
plot_voltage_out.output_backend = 'svg'

plot_voltage_in.yaxis.ticker = [0,1]
plot_voltage_in.xaxis.visible = False
plot_voltage_in.y_range.start = -0.5
plot_voltage_in.y_range.end = 1.5
plot_voltage_in.output_backend = 'svg'

In [ ]:
bokeh.io.show(layout)

## Soft Robot Circuit

### Concentration Sweep

In [ ]:
concentrations = np.logspace(0, 3.5, 20)

filepath_output = './Simulation Datafiles/Robot_Brain.h5'
concentration_averages = {}

with pd.HDFStore(filepath_output) as store:
    for c in concentrations:
        timeseries = store[f'{c}/timeseries']
        parameters = store[f'{c}/parameters']

        mols = 0
        volume_total = 0
        for i in range(1,4):
            volume = parameters.loc[f'actuator_bilayer.volumes[{i}].volume'].value
            volume_total += volume
            mols += timeseries[f'actuator_bilayer.C1[{i}]']*volume
        
        concentration_averages[c] = (mols/volume_total)

        #save memory 
        del timeseries
        del parameters

concentration_averages_frame = pd.DataFrame.from_dict(concentration_averages)


In [ ]:
periods = {}
for c in concentration_averages:
    concentration = concentration_averages[c].loc[10000:]
    
    peak_indices = scipy.signal.find_peaks(concentration, prominence=2)[0]
    # plt.plot(peak_indices, concentration.values[peak_indices], '.')
    # plt.plot(concentration.values, 'orange')
    # plt.show()
    # window = scipy.signal.windows.blackman(concentration_average.shape[-1])
    # concentration_fft = scipy.fft.fft(concentration_average.values*window)
    # concentration_fft_freq = scipy.fft.fftfreq(concentration_average.shape[-1])
    # concentration_fft_period = 1/concentration_fft_freq
    
    peak_times = concentration.index[peak_indices]
    peak_time_deltas = np.diff(peak_times)
    print(peak_time_deltas)

    # Take the average of the penultimate 5 peaks.
    periods[c] = np.mean(np.sort(peak_time_deltas)[-6:-1])

periods = pd.DataFrame.from_dict(periods, orient='index')

In [ ]:
plot_sensor_sensitivity = bokeh.plotting.figure(height = 300, width = 600, x_axis_type = 'log')

plot_sensor_sensitivity.line(x = periods.index, y=periods/60, line_width=2,)
#plot_sensor_sensitivity.add_layout(bokeh.models.Span(location = np.log10(3), dimension = 'height', line_dash = [2, 2]))
#plot_sensor_sensitivity.add_layout(bokeh.models.Span(location = np.log10(30), dimension = 'height', line_dash = [2, 2]))


plot_sensor_sensitivity.output_backend = 'svg'

In [ ]:
bokeh.io.show(plot_sensor_sensitivity)

### Shared Settings

In [ ]:
filepath = './Simulation Datafiles/Robot_Brain_Dynamic.h5'

with pd.HDFStore(filepath) as store:
    timeseries = store[f'timeseries']
    parameters = store[f'parameters']


In [ ]:
colors = bokeh.palettes.Category10_10

time_start = 7060
time_end = 29898

time_min = (timeseries.index - time_start)/60
time_min_end = (time_end-time_start)/60


span_concentration_change_start = bokeh.models.Span(location = (11000-time_start)/60, dimension='height', line_dash = [2, 2])
span_concentration_change_start.level = 'underlay'
span_concentration_change_end = bokeh.models.Span(location = (26000-time_start)/60, dimension='height',  line_dash = [2, 2])
span_concentration_change_end.level = 'underlay'

### Supply

In [ ]:
plot_supply = bokeh.plotting.figure(height = 150, width = 340)

plot_supply.line(x = time_min,
                  y = -timeseries['source_anions.V_delta'],
                  line_width=2,
                  legend_label = 'Cl- Source',
                  color=colors[0]
              )

plot_supply.line(x = time_min,
                  y = timeseries['source_cations.V_delta'],
                  line_width=2,
                  legend_label = 'Na+ Source',
                  color=colors[1],
                  #line_dash = [0, 4],
                  line_cap = 'round',
              )

plot_supply.add_layout(span_concentration_change_start)
plot_supply.add_layout(span_concentration_change_end)

plot_supply.y_range.start = 0
plot_supply.y_range.end = 1.3
plot_supply.x_range.start = 0
plot_supply.x_range.end = time_min_end
plot_supply.legend.location = 'bottom_right'
plot_supply.legend.orientation = 'horizontal'
plot_supply.legend.padding = 5
plot_supply.legend.spacing = 8
plot_supply.output_backend = 'svg'

In [ ]:
bokeh.io.show(plot_supply)

### Oscillator

In [ ]:
plot_oscillator = bokeh.plotting.figure(height = 150, width = 340)


plot_oscillator.line(x = time_min,
                  y = -timeseries['oscillator_b_npn.source_prism.i']/1e-6,
                  line_width=2,
                  legend_label = 'Oscillator L',
                  color=colors[0]
              )

plot_oscillator.line(x = time_min,
                  y = -timeseries['oscillator_a_npn.source_prism.i']/1e-6,
                  line_width=2,
                  legend_label = 'Oscillator R',
                  color=colors[1],
                  line_dash = [1, 3],
                  line_cap = 'round',
              )

plot_oscillator.add_layout(span_concentration_change_start)
plot_oscillator.add_layout(span_concentration_change_end)

#plot_oscillator.y_range.start = 0
plot_oscillator.y_range.end = 2.0
plot_oscillator.x_range.start = 0
plot_oscillator.x_range.end = time_min_end
plot_oscillator.legend.location = 'top_left'
plot_oscillator.legend.orientation = 'horizontal'
plot_oscillator.legend.padding = 5
plot_oscillator.legend.spacing = 8
#plot_oscillator.legend.label_height = 5
plot_oscillator.output_backend = 'svg'

In [ ]:
bokeh.io.show(plot_oscillator)

### Actuator

In [ ]:
colors = bokeh.palettes.Category10_10
plot_actuator = bokeh.plotting.figure(height = 150, width = 340)

plot_actuator.line(x = time_min,
                  y = timeseries['actuator_bilayer.C1[2]'],
                  line_width=2,
                  #legend_label = 'Chlorine Source',
                  color=colors[0]
              )

plot_actuator.add_layout(span_concentration_change_start)
plot_actuator.add_layout(span_concentration_change_end)

plot_actuator.y_range.start = 15
plot_actuator.y_range.end = 60
plot_actuator.x_range.start = 0
plot_actuator.x_range.end = time_min_end
#plot_supply.legend.location = 'bottom_right'
#plot_supply.legend.padding = 5
#plot_supply.legend.spacing = -5
plot_actuator.output_backend = 'svg'

In [ ]:
bokeh.io.show(plot_actuator)

### Sensor

In [ ]:
plot_sensor_input = bokeh.plotting.figure(height = 50, width = 320)

plot_sensor_input.line(x = time_min,
                  y = timeseries['sensor_concentration_world.Ci[1]'],
                  line_width=2,
                  #legend_label = 'Chlorine Source',
                  color=colors[0]
              )


plot_sensor_output = bokeh.plotting.figure(height = 100, width = 320)


plot_sensor_output.line(x = time_min,
                  y = timeseries['sensor_wire_a.i']/1e-9,
                  line_width=2,
                  #legend_label = 'Chlorine Source',
                  color=colors[0]
              )

plot_sensor_output.x_range = plot_sensor_input.x_range


plot_sensor = bokeh.layouts.gridplot(children = [[plot_sensor_input],
                                          [plot_sensor_output]],
                               )

plot_sensor_output.add_layout(span_concentration_change_start)
plot_sensor_output.add_layout(span_concentration_change_end)
plot_sensor_input.add_layout(span_concentration_change_start)
plot_sensor_input.add_layout(span_concentration_change_end)


#plot_sensor.y_range.start = 15
#plot_sensor.y_range.end = 60
plot_sensor_input.yaxis.ticker = [5,50]
plot_sensor_input.xaxis.visible = False

plot_sensor_input.x_range.start = 0
plot_sensor_input.x_range.end = time_min_end
#plot_sensor.legend.location = 'bottom_right'
#plot_sensor.legend.padding = 5
#plot_sensor.legend.spacing = -5

plot_sensor_input.output_backend = 'svg'
plot_sensor_output.output_backend = 'svg'

In [ ]:
bokeh.io.show(plot_sensor)

### Transistor

#### Oscillator

In [ ]:
filepath = './Simulation Datafiles/Robot_Brain_Transistor_Oscillator_Output_Characteristics.h5'

with pd.HDFStore(filepath) as store:
    timeseries = store[f'timeseries']
    parameters = store[f'parameters']

In [ ]:
plot_transistor_output = bokeh.plotting.figure(height = 300, width = 300)

plot_transistor_output.line(x = timeseries['V_ds.v'],
                  y = -timeseries['electrode_source.i']/1e-6,
                  line_width=2,
                  #legend_label = 'Chlorine Source',
                  color=colors[0]
              )

#plot_transistor_output.add_layout(span_concentration_change_start)
#plot_transistor_output.add_layout(span_concentration_change_end)

#plot_transistor_output.y_range.start = 15
#plot_transistor_output.y_range.end = 60
#plot_transistor_output.x_range.start = 0
#plot_transistor_output.x_range.end = time_min_end

plot_transistor_output.output_backend = 'svg'

In [ ]:
bokeh.io.show(plot_transistor_output)

In [ ]:
filepath = './Simulation Datafiles/Robot_Brain_Transistor_Oscillator_Current_Amplification.h5'

with pd.HDFStore(filepath) as store:
    timeseries = store[f'timeseries']
    parameters = store[f'parameters']

In [ ]:
plot_transistor_amplification = bokeh.plotting.figure(height = 300, width = 300)

plot_transistor_amplification.line(x = timeseries['V_b.v'],
                  y = -timeseries['V_b.i']/1e-6,
                  line_width=2,
                  legend_label = 'Base Current',
                  color=colors[0]
              )

plot_transistor_amplification.line(x = timeseries['V_b.v'],
                  y = -timeseries['V_ds.i']/1e-6,
                  line_width=2,
                  legend_label = 'Collector Current',
                  color=colors[1]
              )

plot_transistor_amplification.legend.location = 'top_left'
plot_transistor_amplification.y_range.start = -0.5
plot_transistor_amplification.y_range.end = 5.5
plot_transistor_amplification.x_range.start = 0
plot_transistor_amplification.x_range.end = 0.55
plot_transistor_amplification.output_backend = 'svg'

In [ ]:
bokeh.io.show(plot_transistor_amplification)

#### Actuator

In [ ]:
filepath = './Simulation Datafiles/Robot_Brain_Transistor_Actuator_Output_Characteristics.h5'

with pd.HDFStore(filepath) as store:
    timeseries = store[f'timeseries']
    parameters = store[f'parameters']

In [ ]:
plot_transistor_output = bokeh.plotting.figure(height = 300, width = 300)

plot_transistor_output.line(x = timeseries['V_ds.v'],
                  y = -timeseries['electrode_source.i']/1e-6,
                  line_width=2,
                  #legend_label = 'Chlorine Source',
                  color=colors[0]
              )

#plot_transistor_output.add_layout(span_concentration_change_start)
#plot_transistor_output.add_layout(span_concentration_change_end)

#plot_transistor_output.y_range.start = 15
#plot_transistor_output.y_range.end = 60
#plot_transistor_output.x_range.start = 0
#plot_transistor_output.x_range.end = time_min_end

plot_transistor_output.output_backend = 'svg'

In [ ]:
bokeh.io.show(plot_transistor_output)

In [ ]:
filepath = './Simulation Datafiles/Robot_Brain_Transistor_Actuator_Current_Amplification.h5'

with pd.HDFStore(filepath) as store:
    timeseries = store[f'timeseries']
    parameters = store[f'parameters']

In [ ]:
plot_transistor_amplification = bokeh.plotting.figure(height = 300, width = 300)

plot_transistor_amplification.line(x = timeseries['V_b.v'],
                  y = -timeseries['V_b.i']/1e-6,
                  line_width=2,
                  legend_label = 'Base Current',
                  color=colors[0]
              )

plot_transistor_amplification.line(x = timeseries['V_b.v'],
                  y = -timeseries['V_ds.i']/1e-6,
                  line_width=2,
                  legend_label = 'Collector Current',
                  color=colors[1]
              )

plot_transistor_amplification.legend.location = 'top_left'
plot_transistor_amplification.y_range.start = -0.5
plot_transistor_amplification.y_range.end = 10.5
plot_transistor_amplification.x_range.start = 0
plot_transistor_amplification.x_range.end = 0.55
plot_transistor_amplification.output_backend = 'svg'

In [ ]:
bokeh.io.show(plot_transistor_amplification)

## Wire Enrichment

In [ ]:
filepath = './Simulation Datafiles/Wire_Enrichment.h5'

with pd.HDFStore(filepath) as store:
    timeseries_charged = store[f'timeseries']
    parameters_charged = store[f'parameters']

filepath = './Simulation Datafiles/Wire_Enrichment_Uncharged.h5'

with pd.HDFStore(filepath) as store:
    timeseries_uncharged = store[f'timeseries']
    parameters_uncharged = store[f'parameters']

In [ ]:
line_dash_uncharged = [4, 4]

plot_wire_enrichment_voltage = bokeh.plotting.figure(height = 40, width = 600)

plot_wire_enrichment_voltage.line(x = timeseries_charged.index/60,
                  y = timeseries_charged['source.v'],
                  line_width=2,
                  color='black',
              )

plot_wire_enrichment_current = bokeh.plotting.figure(height = 200, width = 600)

plot_wire_enrichment_current.line(x = timeseries_charged.index/60,
                  y = -timeseries_charged['source.i']/1e-6,
                  line_width=2,
                  legend_label = 'Charged',
                  color='black',
                  line_alpha = 0.6,
              )

plot_wire_enrichment_current.line(x = timeseries_uncharged.index/60,
                  y = -timeseries_uncharged['source.i']/1e-6,
                  line_width=2,
                  legend_label = 'Uncharged',
                  color='black',
                  line_dash = line_dash_uncharged,
              )

for timeseries, parameters in [(timeseries_charged, parameters_charged),
                               (timeseries_uncharged, parameters_uncharged)]:

    baths = ['bath_a', 'bath_b']
    volumes = [1,2,3,4,5]
    concentrations = [1]
    concentration_names = ['salt']
    
    for bath in baths:
        for index_concentration, concentration_name in zip(concentrations, concentration_names):
            timeseries[f'{bath}.{concentration_name}_average'] = 0
    
            volume_total = 0        
            for index_volume in volumes:
                volume = parameters.loc[f'{bath}.volumes[{index_volume}].volume'].value
                concentration = timeseries[f'{bath}.volumes[{index_volume}].C[{index_concentration}]']
                timeseries[f'{bath}.{concentration_name}_average'] += concentration*volume
                volume_total += volume
            timeseries[f'{bath}.{concentration_name}_average']/= volume_total

plot_wire_enrichment_concentration = bokeh.plotting.figure(height = 200, width = 600)

plot_wire_enrichment_concentration.line(x = timeseries_charged.index/60,
                  y = timeseries_charged['bath_a.salt_average'],
                  line_width=2,
                  legend_label = 'Charged, Bath A',
                  color=colors[2],
              )

plot_wire_enrichment_concentration.line(x = timeseries_charged.index/60,
                  y = timeseries_charged['bath_b.salt_average'],
                  line_width=2,
                  legend_label = 'Charged, Bath B',
                  color=colors[4]
              )

plot_wire_enrichment_concentration.line(x = timeseries_uncharged.index/60,
                  y = timeseries_uncharged['bath_a.salt_average'],
                  line_width=2,
                  legend_label = 'Uncharged, Both Baths',
                  color='black',
                  line_dash=line_dash_uncharged,
              )

# plot_wire_enrichment_current.line(x = timeseries['V_b.v'],
#                   y = -timeseries['V_ds.i']/1e-6,
#                   line_width=2,
#                   legend_label = 'Collector Current',
#                   color=colors[1]
#               )

#plot_wire_enrichment_current.legend.location = 'top_left'
#plot_wire_enrichment_current.y_range.start = -0.5
#plot_wire_enrichment_current.y_range.end = 5.5
#plot_wire_enrichment_current.x_range.start = 0
#plot_wire_enrichment_current.x_range.end = 0.55

plot_wire_enrichment_voltage.yaxis.ticker = [0,1]
plot_wire_enrichment_voltage.xaxis.visible = False
plot_wire_enrichment_voltage.output_backend = 'svg'

plot_wire_enrichment_concentration.y_range.start = 0.0
plot_wire_enrichment_concentration.y_range.end = 24
plot_wire_enrichment_concentration.output_backend = 'svg'

plot_wire_enrichment_current.xaxis.visible = False
plot_wire_enrichment_current.output_backend = 'svg'

layout = bokeh.layouts.gridplot(children = [[plot_wire_enrichment_voltage],[plot_wire_enrichment_current],
                                          [plot_wire_enrichment_concentration]],
                               )

bokeh.io.show(layout)

# Tables

In [ ]:
simulation_summary_table = simulation_summaries.copy()

def parse_simulation_options(row):
    options = row['simulationOptions'].split(', ')
    for option in options:
        suboptions = option.split(' = ')
        match suboptions:
            case ['startTime', time_start]:
                ...
            case ['stopTime', time_end]:
                ...
            case ['numberOfIntervals', step_count]:
                ...

    time_simulation = float(time_end)-float(time_start)

    output_dict = {'Simulated Time (s)': time_simulation,
                   'Simulated Step Count': step_count,
                   'Translation Wall Time (s)': row['timeFrontend'] + row['timeBackend'] + row['timeSimCode'] + row['timeTemplates'],
                   'Compilation Wall Time (s)': row['timeCompile'],
                   'Simulation Wall Time (s)': row['timeSimulation'],
                   'Total Wall Time (s)': row['timeTotal'],
                   'Time Ratio': time_simulation/row['timeSimulation']
                  }

    return output_dict
            

simulation_summary_table = simulation_summary_table.apply(parse_simulation_options, result_type='expand', axis='columns') 

table_tex = simulation_summary_table.to_latex()